In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

fecha_hoy = datetime.date.today()
fecha_de_hoy = fecha_hoy.strftime('%d/%m/%Y')

In [2]:
url_base = 'https://cuspide.com/100-mas-vendidos/'    # Paso 1 obtener html
pedido = requests.get(url_base)
html_obtenido = pedido.text                           # Paso 2. html pasarlo a texto ya que si no, nos devuelve un object de requests

soup = BeautifulSoup(html_obtenido, 'html.parser')    # 3. atribuirlo a oup para 'parsear'

In [3]:
h3 = soup.find_all('h3')   # aislamos la etiqueta h3 donde se encuentran el titulo y la url
titulo_obtenido =[]        # obtenemos el titulo

for n in h3:
    titulo = n.find('a').text.strip()
    titulo_obtenido.append(titulo)

In [4]:
url_obtenida = []           # Obtenemos la url
for url in h3:
    u = url.find('a').get('href')
    url_obtenida.append(u)

In [5]:
precio_pesos = []
precio_dolar = []

for e in url_obtenida:   # por cada url, hago un request

      url_bucle = e
      pedido2 = requests.get(url_bucle)
      html = pedido2.text
      sopa_bucle = BeautifulSoup(html, 'html.parser')

      t = sopa_bucle.find('h1').text.strip()

      try:
         # obtenemos el precio en pesos
        precio = sopa_bucle.find('p', class_='price product-page-price')
        precio_texto = precio.text.strip().replace('&nbsp;', '').replace('$', '').replace('\\xa0', '').replace('.', '')
        precio2 = float(precio_texto.replace(',', '.'))
        precio_pesos.append(precio2)

           # **Obtenemos el precio en dólares**
        precio_dolars = sopa_bucle.find('span', style='font-size: 1.3em').text#.strip()#.replace('&nbsp;', '')
        precio_dolars2 = float(precio_dolars.replace(',', '.'))
        precio_dolar.append(precio_dolars2)

        #print(f'se añadio {t} con precios, pesos: {precio2}, dolar: {precio_dolars2} \n')
      except:
         print(f'> El libro: {t}, no se pudo añadir \n')
         precio_pesos.append(None)
         precio_dolar.append(None)

> El libro: ZENSORIALMENTE, no se pudo añadir 

> El libro: HABITOS ATOMICOS, no se pudo añadir 

> El libro: LO MEJOR Y LO PEOR DE INTERNET, no se pudo añadir 

> El libro: LA SOCIEDAD DE LA NIEVE, no se pudo añadir 

> El libro: DEJA DE SER TU, no se pudo añadir 

> El libro: COMO HACER QUE TE PASEN COSAS BUENAS, no se pudo añadir 

> El libro: NUESTRA PARTE DE NOCHE, no se pudo añadir 

> El libro: CUANDO NO QUEDEN MAS ESTRELLAS QUE CONTAR, no se pudo añadir 

> El libro: EL CAMINO DEL ARTISTA, no se pudo añadir 

> El libro: LA PACIENTE SILENCIOSA, no se pudo añadir 

> El libro: LA CASA DE LOS SUICIDIOS, no se pudo añadir 

> El libro: LOS ARBOLES CAIDOS TAMBIEN SON DEL BOSQUE., no se pudo añadir 

> El libro: EL NI/O RESENTIDO, no se pudo añadir 

> El libro: LA INFANCIA DEL MUNDO, no se pudo añadir 

> El libro: 73 MARGARITAS, no se pudo añadir 

> El libro: MUCHAS VIDAS , MUCHOS MAESTROS, no se pudo añadir 

> El libro: EL DIA QUE DEJO DE NEVAR EN ALASKA, no se pudo añadir 

> 

In [6]:
# obtenemos el precio dolar blue
url_blue = 'https://www.clarin.com/economia/divisas-acciones-bonos?gclid=Cj0KCQiA-62tBhDSARIsAO7twbYBCdmDyspJS1SAuZ8OcXFZZKpEUJx-A93vXK2IlRpKwbhC9_-eEAIaAk6IEALw_wcB'
pedido3 = requests.get(url_blue)
html3 = pedido3.text
sopa_blue = BeautifulSoup(html3, 'html.parser')

blue = sopa_blue.find_all('div', class_='selling')
blues = []

for n in blue:
    num = n.find('span').text.strip()
    blues.append(num)

blue_verdadero = float(blues[2].replace('$', ''))  # por fin el dolar blue

In [7]:
precio_blue = []

for p in range(len(precio_pesos)):
    if precio_pesos[p] != None:
        precio_blue.append(round(precio_pesos[p]/blue_verdadero, 2))
    else:
        precio_blue.append(None)

In [8]:
# creamos un dataframe a partir de un diccionario
d= dict( titulo = titulo_obtenido,
        url = url_obtenida,
        precio_pesos = precio_pesos,
        precio_dolar = precio_dolar,
        precio_dolar_blue = precio_blue,
        fecha = fecha_de_hoy)

df = pd.DataFrame(d)
df.to_csv('top_cien_cuspide.csv', sep='|', encoding='utf-8', index=False)